In [35]:
import numpy as np
import numba
import tqdm

In [36]:
test_input = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

In [37]:
def parse_input(data):
    # numarr = np.array([np.array([0 if x == "." else 1 for x in row]) for row in data.split("\n")])
    arr = np.array([np.array([x for x in row]) for row in data.split("\n")])
    return arr

In [47]:
def rotate_direction(direction):
    return np.array([direction[1], -direction[0]])

In [39]:
def step(current_loc, direction, data):
    next_loc = current_loc+direction

    #if we move out of bounds
    if any(next_loc) < 0 or any(next_loc > data.shape):
        return next_loc, direction
    #if next loc is obstacle
    if data[tuple(next_loc)] == "#":
        turns = 0
        while data[tuple(next_loc)] == "#":
            direction = rotate_direction(direction)
            next_loc = current_loc
            turns += 1
            if turns > 4: 
                raise Exception("STUCK!")           
    
    return next_loc, direction

In [40]:
def walk(puzzle_data):
    data = parse_input(puzzle_data)
    # print(data)
    h, w = data.shape
    guard = np.argwhere(data=="^")[0]
    direction = np.array([-1,0])
    path = np.zeros(data.shape)
    path[tuple(guard)] = 1
    #while guard on the map:
    print(guard)
    while all(guard > 0) and all(guard < (np.array(data.shape) - 1)):
        path[tuple(guard)] = 1
        next_loc, direction = step(guard, direction, data)
        guard = next_loc
        
    
    # print(path)
    return(np.sum(path>0)+1)
    
        
        
        

In [41]:
walk(test_input)

[6 4]


np.int64(41)

In [42]:
with open("puzzle_input.txt", "r") as f:
    puzzle_input = f.read()

walk(puzzle_input)

[60 60]


np.int64(4663)

## part two:
amend our walk function to log locations and directions. if we are in a location and direction we have been in before, we are in a loop. 

In [57]:

def step(current_loc, direction, data):
    next_loc = current_loc+direction

    #if we move out of bounds
    if any(next_loc) < 0 or any(next_loc > data.shape):
        return next_loc, direction
    #if next loc is obstacle
    if data[tuple(next_loc)] == "#":
        turns = 0
        while data[tuple(next_loc)] == "#":
            direction = rotate_direction(direction)
            next_loc = current_loc
            turns += 1
            if turns > 4: 
                raise Exception("STUCK!")           
    
    return next_loc, direction

def walk(data, start_spot):
    guard = start_spot
    direction = np.array([-1,0])
    path = np.zeros(data.shape)
    path[tuple(guard)] = 1
    been_there = set()
    been_there.add((tuple(guard), tuple(direction)))
    #while guard on the map:
    history = [[guard, direction]]
    while all(guard > 0) and all(guard < (np.array(data.shape) - 1)):
        path[tuple(guard)] = 1
        next_loc, direction = step(guard, direction, data)
        if (tuple(next_loc), tuple(direction)) in been_there:
            return path, True
        guard = next_loc
        been_there.add((tuple(guard), tuple(direction)))
    #final path update
    path[tuple(guard)] = 1
        
    
    # print(path)
    return path, False
    

In [58]:
#test by placing an object next to the start spot
test_data = parse_input(test_input)
start_spot = np.argwhere(test_data=="^")[0] 
object_spot =start_spot+ np.array([0,-1])
new_map = test_data.copy()
new_map[tuple(object_spot)] = "#"
p, c = walk(new_map, start_spot)

In [59]:
def place_obstructions(data):
    #find the original path
    start_spot = np.argwhere(data=="^")[0]
    orig_path, is_cycle = walk(data, start_spot=start_spot)
    #find inceces of the path. here we can place objects
    places = np.argwhere(orig_path!=0)
    good_places = 0
    #for every place we can place an object:
    print(len(places))
    for place in places:
        #copy the map
        new = data.copy()
        #place an object
        new[tuple(place)] = "#"
        #walk the path
        p, cycle = walk(new, start_spot)
        if cycle:
            good_places+=1
    return good_places
        

In [60]:
%%time
data = parse_input(puzzle_input)


place_obstructions(data)

4663
CPU times: user 2min 16s, sys: 29.1 ms, total: 2min 16s
Wall time: 2min 17s


1530

That is nice and all but should be faster....

In [61]:
from itertools import cycle

In [76]:
direction = np.array([1, 0])
directions = [direction]
for i in range(3):
    directions.append(rotate_direction(directions[-1]))
directions = cycle(directions)

In [78]:
direction = next(directions)

In [81]:
direction

array([1, 0])

In [82]:
direction

array([1, 0])